In [1]:
import sys


In [3]:
sys.argv.pop()

'-f'

In [4]:
sys.argv

['/Users/aciuffo/Virtualenvs/python2/lib/python2.7/site-packages/ipykernel/__main__.py']

In [7]:
from __future__ import print_function
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

from apiclient import errors as gerror

SCOPES = 'https://www.googleapis.com/auth/drive'
# SCOPES = 'https://www.googleapis.com/auth/drive.file' # try this one!
# SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret_qs.json'
APPLICATION_NAME = 'Drive API Python Quickstart'

def getCredentials():
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
        credentials = tools.run_flow(flow, store, None)
    return(credentials)






In [9]:
credentials = getCredentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', credentials=credentials)

In [63]:
class gDrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        self.service = object
    
    def quote(self, string):
        '''
        add double quotes arounda string
        '''
        return('"'+str(string)+'"')
    
    def addFile(self, name = None, folder = False):
        pass
    
    
    def search(self, name = None, trashed = False, fuzzy = False, parent = None):
        if name is None:
            print ('specify a folder or file name')
            return(False)
        else:
            # this needs to be surrounded by double quotes
            name = self.quote(name)
                
        # give a reasonable value if this is null
        if parent:
            parentVal = self.quote(parent)            
        else:
            parentVal = self.quote('null')

            
        if fuzzy:
            fuzzyVal = ' contains '
        else:
            fuzzyVal = '='
            
        supportedFeatures = {'name':'name'+fuzzyVal+self.quote(eval(name)), 
                             'trashed': 'trashed='+eval(self.quote(trashed)), 
                             'parent': self.quote(eval(parentVal))+' in parents'}
        
        # exclude these if they are set to none or false
        nullFeatures =['parent']
        
        queryList = []
        for each in supportedFeatures:
            if each in nullFeatures and not eval(each):
                pass
            else:
                queryList.append(str(supportedFeatures[each])) #+ str(myDict[each]))

        query = ' and '.join(queryList)        
        print(query)
        
        try:
            result = self.service.files().list(q=query).execute()
        except gerror.HttpError as e:
            print('File Not Found')
            return(False)
        
        return(result)

In [64]:
service.files().list(q='name contains "more" and trashed=False').execute()

{u'files': [{u'id': u'0B9WTleJ1MzaYaEk1akx4WEFtMFU',
   u'kind': u'drive#file',
   u'mimeType': u'application/vnd.google-apps.folder',
   u'name': u'more cows'},
  {u'id': u'0B9WTleJ1MzaYVkRscmdJU3dlUXM',
   u'kind': u'drive#file',
   u'mimeType': u'application/vnd.google-apps.folder',
   u'name': u'Moreno, Manuel - 504155'},
  {u'id': u'0B5r-H8QVHwi8OUI0T3IyMDRmdWM',
   u'kind': u'drive#file',
   u'mimeType': u'text/xml',
   u'name': u'https-::drive.googleblog.com:2013:10:more-magic-in-slides-editable-themes.html.webloc'},
  {u'id': u'0B4DkefTpSUUmeFhNT0JVOHJIeWs',
   u'kind': u'drive#file',
   u'mimeType': u'image/png',
   u'name': u'More Tools.png'},
  {u'id': u'0B4DkefTpSUUmd3RGdUdxUDBZVTg',
   u'kind': u'drive#file',
   u'mimeType': u'image/png',
   u'name': u'More Tools.png'},
  {u'id': u'0B96rPy38vy6Ac1o3NmVveWQxWTQ',
   u'kind': u'drive#file',
   u'mimeType': u'text/xml',
   u'name': u'http-::appleinsider.com:articles:15:06:12:inside-ios-9-apples-notes-app-is-greatly-enhanced-w

In [67]:
myDrive = gDrive(service)
myDrive.search("cows", fuzzy = True)

name contains "cows" and trashed=False


3

In [68]:
myDrive.search(name = "more cows")

name="more cows" and trashed=False


{u'files': [{u'id': u'0B9WTleJ1MzaYaEk1akx4WEFtMFU',
   u'kind': u'drive#file',
   u'mimeType': u'application/vnd.google-apps.folder',
   u'name': u'more cows'}],
 u'incompleteSearch': False,
 u'kind': u'drive#fileList'}